In [3]:
import pandas as pd
pd.options.display.max_columns = None # display all the columns

import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# load dataset

df_original = pd.read_csv('Data_features.csv') # feature set
obs = pd.read_csv('Data_train_set.csv') # User ID, Churn
est = pd.read_csv('Data_test_set.csv')  # User ID to predict Churn
obs_id = obs['ID'].values # IDs in factual data
est_id = est['ID'].values # IDs to analyse
df = df_original[df_original.ID.isin(obs_id)] # df for training

dfs = []
for i in range(1, 7):
    dfs.append(df[df.MONTH_NUM_FROM_EVENT == i]) # the i month data
    print(i)

dfs_col_ren = []
for i in range(6):
    df = dfs[i].drop(['ID', 'MONTH_NUM_FROM_EVENT'], axis=1)
    d = { e: str(i+1) + e for e in list(df.columns)}
    df = df.rename(index=str, columns=d)
    dfs_col_ren.append(df)

df_uni = dfs_col_ren[0]
for i in range(1,6):
    df_uni = pd.concat([df_uni.reset_index(drop=True), dfs_col_ren[i].reset_index(drop=True)], axis=1)

df_trans = df_uni # dfs_col_ren[5] # df_uni

X = np.array(df_trans.values) # array of features from 
y = np.array(obs.TARGET)
from sklearn.utils.validation import assert_all_finite
nanplaces = np.isnan(X)
X[nanplaces] = 0.01
# run checks
print(assert_all_finite(X))
# should output None
print(np.any(np.isnan(X)), np.all(np.isfinite(X))) 
# should output False True

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import VotingClassifier

clf1 = LogisticRegression(C=0.02, solver='liblinear', max_iter=4000)
clf2 = LogisticRegression(C=0.02, solver='liblinear', max_iter=4000)

eclf1 = VotingClassifier(estimators=[('lr1', clf1), ('lr2', clf2)], voting='soft')
eclf1 = eclf1.fit(X, y)


1
2
3
4
5
6
None
False True


In [5]:
df = df_original[df_original.ID.isin(est_id)] # df for testing

dfs = []
for i in range(1, 7):
    dfs.append(df[df.MONTH_NUM_FROM_EVENT == i]) # the i month data
    print(i)
    
dfs_col_ren = []
for i in range(6):
    df = dfs[i].drop(['ID', 'MONTH_NUM_FROM_EVENT'], axis=1)
    d = { e: str(i+1) + e for e in list(df.columns)}
    df = df.rename(index=str, columns=d)
    dfs_col_ren.append(df)
    # display(df.head(), df.shape)

df_uni = dfs_col_ren[0]
for i in range(1,6):
    df_uni = pd.concat([df_uni.reset_index(drop=True), dfs_col_ren[i].reset_index(drop=True)], axis=1)

# display(df_uni.head(), df_uni.shape)

df_trans = df_uni # dfs_col_ren[5] # df_uni

X = np.array(df_trans.values) # array of features from 

from sklearn.utils.validation import assert_all_finite
nanplaces = np.isnan(X)
X[nanplaces] = 0.01
# run checks
print(assert_all_finite(X))
# should output None
print(np.any(np.isnan(X)), np.all(np.isfinite(X))) 
# should output False True

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

y_prob_ = eclf1.predict_proba(X)

est['TARGET'] = pd.Series(y_prob_[:,1], index=est.index)
# results_sample = pd.read_csv('Sample_test.txt')
est.to_csv('YanushevskyiArtemii_test.txt', index=False)

1
2
3
4
5
6
None
False True
